In [175]:
#!pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [176]:
#!pip list

In [8]:
import tensorflow as tf
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [10]:
tf.config.list_physical_devices('GPU')

[]

# Removing unclear images

In [180]:
import cv2
import imghdr

In [12]:
data_dir = 'augmented' 

In [182]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [183]:
os.listdir(data_dir)

['crab', 'dolphin', 'seaturtle', 'starfish']

In [184]:
for image_class in os.listdir(data_dir): 
    print(image_class)

crab
dolphin
seaturtle
starfish


In [185]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# Load Data

# Load Data

In [186]:
import numpy as np
#from matplotlib import pyplot as plt

In [41]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 20152 files belonging to 4 classes.


In [30]:
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

# Scaling Data

In [42]:
#Returns value between 0 and 1
data = data.map(lambda x,y: (x/255, y))

# Split Data

In [190]:
len(data)

630

In [43]:
#split into train and test

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [33]:
print(train_size)
print(val_size)
print(test_size)

441
126
64


In [44]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# Deep Learning Model using CNN

In [194]:
train

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [195]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [196]:
model = Sequential()

In [197]:
#Add convolution and max-pooling layer
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [198]:
model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [199]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 127, 127, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 30, 30, 16)      

# Training

In [200]:
logdir='logs'

In [201]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [202]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
441/441 [==============================] - 376s 849ms/step - loss: 0.5683 - accuracy: 0.7713 - val_loss: 0.0968 - val_accuracy: 0.9779
Epoch 2/20
441/441 [==============================] - 434s 983ms/step - loss: 0.1233 - accuracy: 0.9653 - val_loss: 0.0721 - val_accuracy: 0.9750
Epoch 3/20
441/441 [==============================] - 556s 1s/step - loss: 0.0894 - accuracy: 0.9822 - val_loss: 0.0356 - val_accuracy: 0.9906
Epoch 4/20
441/441 [==============================] - 503s 1s/step - loss: 0.0689 - accuracy: 0.9887 - val_loss: 0.2388 - val_accuracy: 0.9340
Epoch 5/20
441/441 [==============================] - 481s 1s/step - loss: 0.0597 - accuracy: 0.9897 - val_loss: 0.0312 - val_accuracy: 0.9923
Epoch 6/20
441/441 [==============================] - 474s 1s/step - loss: 0.0480 - accuracy: 0.9931 - val_loss: 0.0281 - val_accuracy: 0.9955
Epoch 7/20
441/441 [==============================] - 511s 1s/step - loss: 0.0882 - accuracy: 0.9889 - val_loss: 0.0185 - val_accuracy: 

# Performance Plot

In [203]:
#fig = plt.figure()
#
#plt.plot(hist.history['loss'], color='teal', label='loss')
#plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
#
#fig.suptitle('Loss', fontsize=20)
#
#plt.legend(loc="upper right")
#plt.show()

In [204]:
#fig = plt.figure()
#
#plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
#plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
#
#fig.suptitle('Accuracy', fontsize=20)
#
#plt.legend(loc="lower right")
#plt.show()

# Evaluation

In [50]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [51]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [40]:
import numpy as np
from sklearn.metrics import r2_score

for batch in test.as_numpy_iterator():
    cmp = [] 
    X, y = batch
    yhat = model.predict(X)

    for j in range(len(yhat)):
        cmp.append(np.argmax(yhat[j]))
    
    print(y)
    print(cmp)

    print(r2_score(y,cmp))

    pre.update_state(y, cmp)
    re.update_state(y, cmp)
    acc.update_state(y, cmp)

1/1 [==============================] - 0s 225ms/step
[3 3 2 2 1 1 2 0 2 1 1 3 0 2 0 2 2 1 0 2 1 1 0 2 1 3 2 1 0 3 1 2]
[3, 3, 2, 2, 1, 1, 2, 0, 2, 1, 1, 3, 0, 2, 0, 2, 2, 1, 0, 2, 1, 1, 0, 2, 1, 3, 2, 1, 0, 3, 1, 2]
1.0


AttributeError: 'numpy.float64' object has no attribute 'update_state'

In [53]:
print(f"precision : {pre.result().numpy()}")
print(f"Recall : {re.result().numpy()}")
print(f"accuracy : {acc.result().numpy()}")

precision : 1.0
Recall : 1.0
accuracy : 1.0


# Test

In [209]:
#img = cv2.imread('data/seaturtle/image_300.jpeg')
#plt.imshow(img)
#plt.show()

In [210]:
#resize = tf.image.resize(img, (256,256))
#plt.imshow(resize.numpy().astype(int))
#plt.show()

In [211]:
#yhat = model.predict(np.expand_dims(resize/255, 0))

In [212]:
#yhat = yhat.round()
#pval = np.argmax(yhat)

#print(yhat)
#print(pval)

In [213]:
#if(pval == 0): 
#    print(f'Predicted class is {catgry[0]}')
#elif(pval == 1):
#    print(f'Predicted class is {catgry[1]}')
#elif(pval == 2):
#    print(f'Predicted class is {catgry[2]}')
#elif(pval == 3):
#    print(f'Predicted class is {catgry[3]}')
#elif(pval == 4):
#    print(f'Predicted class is {catgry[4]}')
#elif(pval == 5):
#    print(f'Predicted class is {catgry[5]}')
#elif(pval == 6):
#    print(f'Predicted class is {catgry[6]}')
#elif(pval == 7):
#    print(f'Predicted class is {catgry[1]}')

# Save the Model

In [37]:
from tensorflow.keras.models import load_model

In [215]:
model.save(os.path.join('models','imageclassifier.h5'))

In [47]:
new_model = load_model(os.path.join('models','imageclassifier.h5'))

In [217]:
#new_model.predict(np.expand_dims(resize/255, 0))